In [9]:
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import backend as K

In [28]:
# In 'th' mode, the channels dimension (the depth) is at index 1
K.set_image_dim_ordering('th')

np.random.seed(1000)

random_dim = 100

batch_size = 128

epochs = 10
adam = Adam(lr=0.0002, beta_1=0.5) 

In [3]:
# Load MNIST data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = X_train[:, np.newaxis, :, :]

In [22]:
# Generator
generator = Sequential()
generator.add(Dense(128 * 7 * 7, input_shape=(random_dim,), kernel_initializer='random_uniform'))
generator.add(LeakyReLU(0.2))
generator.add(Reshape((128, 7, 7)))
generator.add(UpSampling2D(size=(2, 2), data_format='channels_first'))
generator.add(Conv2D(64, kernel_size=(5, 5), data_format='channels_first', padding='same'))
generator.add(LeakyReLU(0.2))
generator.add(UpSampling2D(size=(2, 2), data_format='channels_first'))
generator.add(Conv2D(1, kernel_size=(5, 5), data_format='channels_first', padding='same', activation='tanh'))
generator.compile(loss='binary_crossentropy', optimizer=adam)

In [16]:
# Discriminator
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=(5, 5), strides=(2, 2), data_format='channels_first', padding='same', kernel_initializer='random_uniform', input_shape=(1, 28, 28)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(64, kernel_size=(5, 5), strides=(2, 2), data_format='channels_first', padding='same', kernel_initializer='random_uniform')) 
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer=adam)

In [24]:
# Combined network
discriminator.trainable = False
gan_input = Input(shape=(random_dim,))
x = generator(gan_input)
gan_output = discriminator(x)
gan = Model(inputs=gan_input, outputs=gan_output)
gan.compile(loss='binary_crossentropy', optimizer=adam)

In [29]:
total_batch_number = X_train.shape[0] / batch_size

print 'Number of epochs: ', epochs
print 'Batch size: ', batch_size
print 'Batches per epoch: ', total_batch_number



Number of epochs:  10
Batch size:  128
Batches per epoch:  468


In [ ]:
for e in xrange(1, epochs + 1):
    print '-' * 15, 'Epoch %d' % e, '-' * 15

    for _ in tqdm(xrange(total_batch_number)):

        # Get a random set of input images
        batch_indices = np.random.randint(0, X_train.shape[0], size=batch_size)
        real_images = X_train[batch_indices]

        # Generate fake MNIST images
        noise = np.random.normal(0, 1, size=[batch_size, random_dim])
        generated_images = generator.predict(noise)
        
        X = np.concatenate([real_images, generated_images])

        # Labels for generated and real data
        yDis = np.zeros(2 * batch_size)
        # One-sided label smoothing
        yDis[:batch_size] = 0.9

        # Train discriminator
        discriminator.trainable = True
        d_loss = discriminator.train_on_batch(X, yDis)

        # Train generator
        noise = np.random.normal(0, 1, size=[batch_size, random_dim])
        yGen = np.ones(batch_size)
        discriminator.trainable = False
        g_loss = gan.train_on_batch(noise, yGen)
